In [3]:
import cv2
import torch
torch.cuda.empty_cache()
from PIL import Image
import numpy as np
import os
import time
import glob
import numpy as np
# Model
model = torch.hub.load('ultralytics/yolov5', 'custom', path_or_model='last.pt')  # custom model
model.conf = 0.4

Using cache found in C:\Users\CIEDEV/.cache\torch\hub\ultralytics_yolov5_master

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Focus                     [3, 32, 3]                    
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  1    156928  models.common.C3                        [128, 128, 3]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  1    625152  models.common.C3                        [256, 256, 3]                 
  7                -1  1   1180672  models.common.Conv         

In [192]:


dirr = 'E:/images/right_view'    ############################### USER INPUT

save_img_dir = r'C:\Users\CIEDEV\Desktop\Nabin\Saved'  #########################     USERINPUT

files = glob.glob(dirr + '/**/*.tiff', recursive=True)[:200]  #--> ### UserInput

img_name = [ os.path.basename(img) for img in files]

In [193]:
dh = 1200
dw = 1920

result = model(files)  # applying model to img
detection = result.xyxy  # coordinates of 2/3 bottles each image 

final_coords = []           #coords for the middle image
for det in detection:
    coords = det[:,:4].cpu().numpy()
    print(coords)
    if len(coords) == 1:     ## code to identify center image and append to the final coord
         final_coords.append(coords)
            
    elif len(coords) == 2:
        
        y = [coords[0][1],coords[1][1]]
        if coords[0][1] > 500.0 or coords[1][1] > 500.0:
            sortt = np.argsort(y)==0
            index = [i for i, x in enumerate(sortt) if x]
            final_coords.append(coords[index])
            
            
            
        elif coords[0][1] < 500.0 or coords[1][1] < 500.0:
            sortt = np.argsort(y)==1
            index = [i for i, x in enumerate(sortt) if x]
            final_coords.append(coords[index])
            
            
        
             
    elif len(coords) == 3:                        
        y = [coords[0][1],coords[1][1],coords[2][1]]
        sortt = np.argsort(y)==1
        index = [i for i, x in enumerate(sortt) if x]
        final_coords.append(coords[index])


for index, coord in enumerate(final_coords):  
    img = files[index] 
    name_img = img_name[index]
   
   
   
    try:
        crop_img = img[int(coord[0][1])-5:int(coord[0][3])+5, 0: dw] 
        os.chdir(save_img_dir)  # Save image to directory mentioned
        cv2.imwrite(name_img, crop_img)
    except Exception:
        pass
   
print('Index: {}'.format(index+1))
    

[[          0         309        1857       886.5]
 [          0           0        1842         438]
 [          0       796.5      1918.5        1200]]
[[          0      331.88        1887         891]
 [          0           0        1854      454.12]
 [       37.5      809.25      1582.5      1182.8]]
[[          0      314.81        1866      902.25]
 [          0           0        1821      461.62]
 [       58.5      824.25        1560      1194.8]]
[[        105      347.62      1690.5       838.5]
 [          0           0        1881      446.62]
 [     108.75      815.25        1698      1197.8]]
[[      157.5           0      1636.5         411]
 [          0      356.62        1851         870]
 [      54.75      837.75        1560      1175.2]]
[[          0           0        1803      444.75]
 [         72      834.75      1552.5      1178.2]
 [          0      333.38        1851       901.5]]
[[          0      294.38        1911       907.5]
 [          0           0

Index: 200
